# The ID3 Algorithm

**ID3 (Iterative Dichotomizer 3)**: goes through every feature to find the most valuable attribute, then splits based on that attribute. Moves down the tree until it either has a pure class or has met a terminating condition


**Requirements for ID3:**
- outcomes must be binary
- attributes used to build the tree must be categorical (and can have many categories)
- categories must be known and countable

**Entropy formula** This is the definition for Shannon Entropy. 
$$ H = \sum_{i=1}^n P(x_i)log_2 \frac{1}{P(x_i)} $$


## Calculating Entropy 

Example: Let's say we have 20 students, and we're trying to classify them as male and female. The only attribute we have is whether their height is tall, medium, or short.

12 boys
- 4 tall
- 6 medium
- 2 short

8 girls
- 1 tall
- 2 medium
- 5 short

What is the entropy, both before any rule is applied and then after applying a rule for being tall?



The initial entropy is just the formula plugged in over both the possible classes of interest:

$$ H = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$


$$ = \frac{12}{20}*log_2 \frac{20}{12} + \frac{8}{20}*log_2 \frac{20}{8} = .971 $$


What if we apply the rule _height = short_? We need to calculate the weighted average of the two entropies, one for the short students and one for the non-short students.

$$ H(short) = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$

$$ = \frac{2}{7}*log_2 \frac{7}{2} + \frac{5}{7}*log_2 \frac{7}{5} = .863 $$

$$ H(not\_short) = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$

$$ = \frac{10}{13}*log_2 \frac{13}{10} + \frac{3}{13}*log_2 \frac{13}{3} = .779 $$

Note that all the probabilities here are conditional on the criteria we're assuming (short or not short). Now the weighted average of the two is just:

$$ P(short) * H(short) + P(not\_short) * H(not\_short) = \frac{7}{20} * .863 + \frac{13}{20} * .779 = .809 $$

So our entropy from this question would go from .971 to .809. That's an improvement. Use the space below to calculate the entropy of the other criteria, for we asked whether the students were tall or medium.

In [1]:
from math import log2

#Calculate entropy for tall and medium
## First, medium:

H_med = (6/8) * (log2(8/6)) + (2/8) * (log2(8/2))
H_not_med = (6/12) * log2(12/6) + (6/12) * (log2(12/6))
H_med_w = (8/20) * H_med + (12/20) * H_not_med
print('Rule: height = medium\n',H_med_w)

#tall
H_tall = (4/5) * (log2(5/4)) + (1/5) * (log2(5/1))
H_not_tall = (8/15) * (log2(15/8)) + (7/15) * (log2(15/7))
H_tall_w = (5/20) * H_tall + (15/20) * H_not_tall
print('\nRule: height = tall\n',H_tall_w)

Rule: height = medium
 0.9245112497836532

Rule: height = tall
 0.9280757477080679



Which entropy offers the most information gain? Short

## Pseudocoding the Algorithm

**Pseudocode** is the processes of writing the steps and logic you would implement in code, but in normal language rather than commands a programming language could execute. It can be a useful way to chart out how you want to build an algorithm and is a common topic for technical interviews. Here we'll use pseudocode to explain the ID3 algorithm.

Here is reasonable pseudocode for ID3, which we'll then follow up with an explanation of the steps. The outcome for this variable will be A or B. An attribute is denoted a<sub>i</sub>. A value of that attribute is v<sub>i</sub>.


<pre>
Algorithm(Observations, Outcome, Attributes)
    Create a root node.
    If all observations are 'A', label root node 'A' and return.
    If all observations are 'B', label root node 'B' and return.
    If no attributes return the root note labeled with the most common Outcome.
    Otherwise, start:
        For each value v<sub>i</sub> of each attribute a<sub>i</sub>, calculate the entropy.
        The attribute a<sub>i</sub> and value v<sub>i</sub> with the lowest entropy is the best rule.
        The attribute for this node is then a<sub>i</sub>
            Split the tree to below based on the rule a<sub>i</sub> = v<sub>i</sub>
            Observations<sub>v<sub>i</sub></sub> is the subset of observations with value v<sub>i</sub>
            If Observations<sub>v<sub>i</sub></sub> is empty cap with node labeled with most common Outcome
            Else at the new node start a subtree (Observations<sub>v<sub>i</sub></sub>, Target Outcome, Attributes - {a<sub>i</sub>}) and repeat the algorithm
</pre>

Now let's walk through this pseudocode algorithm in plain English piece by piece.

First you create a root node. Simple enough, you have to start with something.

The next two lines say that if you're already exclusively one class, just label with that class and you're done. Similarly the following line says that if you don't have any attributes left you're also done, labeling with the most common outcome.

Then we get into the _real_ algorithm. First you have to find the best attribute by calculating entropies for all possible values. The attribute value pair with the lowest entropy is then the best attribute and the attribute you give to the node.

You use that rule to split the node, creating subsets of observations. There are then two new nodes, each with a subset of the observations corresponding to the rule. If obsevations are null then label with the dominant outcome.

Otherwise at each new node start the algorithm again.

This is how a decision tree would actually function. Understanding this should give you some insight into how this algorithm works and reveals several attributes of the algorithm. Firstly, the solution is not necessarily optimal. The tree can get stuck in local optima, just like with the gradient descent algorithm. It also has no way to work backwards if it finds itself in an informationless space. You can add criteria to make it stop before the tree has grown to run out of attributes or all leaf nodes are single classes.